In [1]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/My Drive/dp_tomastik/code')
    !bash "/content/drive/My Drive/dp_tomastik/code/scripts/install_libs.sh"
    
import SimpleITK as sitk
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import pandas as pd
import logging
import datetime
from torchio import RandomAffine, Compose, ZNormalization
from operator import itemgetter
from IPython.display import display, Markdown
from ipywidgets import widgets

from src.helpers import preview_3d_image
from src.helpers import show_cuda_usage, preview_model_dataset_pred, preview_dataset
from src.helpers import get_threshold_info_df, get_rescaled_preds
from src.helpers import compare_prediction_with_ground_true, compare_one_prediction_with_ground_true
from src.helpers import get_img_outliers_pixels, get_raw_with_prediction
from src.helpers import get_rescaled_pred
from src.helpers import get_transformed_label_np, create_regis_trans_list, trans_list

from src.dataset import HaNOarsDataset, transform_input_with_registration, get_norm_transform
from src.dataset import get_full_res_cut, get_cut_lists, OARS_LABELS, get_dataset, get_dataset_info, get_dataset_transform
from src.dataset import split_dataset, copy_split_dataset

from src.model_and_training import prepare_model, train_loop, show_model_info, load_checkpoint_model_info
from src.model_and_training import iterate_model_v3v2
from src.model_and_training.getters.get_device import get_device
from src.model_and_training.architectures.unet_architecture_v3v2 import UNetV3v2

from src.consts import DATASET_MAX_BOUNDING_BOX, DESIRE_BOUNDING_BOX_SIZE
  
torch.manual_seed(20)
logging.basicConfig(filename='logs/model3v2_all_organs_jupyter.log', level=logging.DEBUG)

print('Dataset biggest bounding box wihtout spinal cord', DATASET_MAX_BOUNDING_BOX)
print('Cut target size', DESIRE_BOUNDING_BOX_SIZE)
print('Done Init')

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits

Dataset biggest bounding box wihtout spinal cord [56, 177, 156]
Cut target size [72, 192, 168]
Done Init


# Training all organs models

training each model

In [2]:
# showing dataset input example
# data_path = f'./data/HaN_OAR_cut_left_parotid_reg'
# cut_full_res_dataset = HaNOarsDataset(data_path, size=50, load_images=False)
# cut_full_res_dataset.load_from_file(data_path)
# cut_full_res_dataset.set_output_label(OARS_LABELS.PAROTID_GLAND_R)
                                     
# preview_3d_image(cut_full_res_dataset[0][0][0], figsize=(4,4))
# preview_3d_image(cut_full_res_dataset[0][0][1], figsize=(4,4))
# preview_3d_image(cut_full_res_dataset[0][1], figsize=(4,4))

In [3]:
filter_labels = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels['SPINAL_CORD']

tmp_list = list(filter_labels.items())
labels_list = list()

# creating registration dataset and organ segmentation pairs
# dataset use inverted labeling of left and right
labels_list.append(('left_parotid', *tmp_list[12]))
labels_list.append(('right_parotid', *tmp_list[11]))

labels_list.append(('brainstem', *tmp_list[10]))
labels_list.append(('parotids', *tmp_list[10]))
labels_list.append(('all_maps', *tmp_list[10]))

for DATASET_REG_NAME, OAR_KEY, OAR_VALUE in labels_list:
    print(f"{DATASET_REG_NAME}, {OAR_KEY}, {OAR_VALUE}")

left_parotid, PAROTID_GLAND_R, 13
right_parotid, PAROTID_GLAND_L, 12
brainstem, PITUITARY, 11
parotids, PITUITARY, 11
all_maps, PITUITARY, 11


In [ ]:
TRAIN_MODELS = True
if TRAIN_MODELS:
    for DATASET_REG_NAME, OAR_KEY, OAR_VALUE in labels_list:
        # loading dataset
        data_path = f'./data/HaN_OAR_cut_{DATASET_REG_NAME}_reg'
        cut_full_res_dataset = HaNOarsDataset(data_path, size=50, load_images=False)
        cut_full_res_dataset.load_from_file(data_path)
        cut_full_res_dataset.set_output_label(OAR_VALUE)
        
        cut_full_res_dataset_obj = split_dataset(cut_full_res_dataset, train_size=40, valid_size=5, test_size=5)
        cut_train_dataset, cut_valid_dataset, cut_test_dataset = itemgetter(*['train_dataset', 'valid_dataset', 'test_dataset'])(cut_full_res_dataset_obj)
        
        # preparing model name
        log_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        model_name = f'{log_date}_3d_unet_lowres_model3v2__cloud-{OAR_KEY}-{DATASET_REG_NAME}_reg'

        print(f'Training model with dataset label \'{OAR_KEY}\', value \'{OAR_VALUE}\'')
        print(f'folder \'{model_name}\'')
        cut_model_info = prepare_model(epochs=100,
                                       learning_rate=3e-4,
                                       in_channels=8,
                                       input_data_channels=1,
                                       output_label_channels=1,
                                       dropout_rate=0.2,
                                       train_batch_size=2,
                                       model_name=model_name,
                                       train_dataset=cut_train_dataset, 
                                       valid_dataset=cut_valid_dataset, 
                                       test_dataset=cut_test_dataset,
                                       model_class=UNetV3v2)
        show_model_info(cut_model_info)
        print('\n\n')
        train_loop(cut_model_info, iterate_model_fn=iterate_model_v3v2)
        print('\n\n')

        # clearing memory
        torch.cuda.empty_cache()

# Evaluating Model

In [ ]:
filter_labels = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels['SPINAL_CORD']

tmp_list = list(filter_labels.items())
labels_list = list()

# creating registration dataset and organ segmentation pairs
# dataset use inverted labeling of left and right
labels_list.append(('left_parotid', *tmp_list[12]))
labels_list.append(('right_parotid', *tmp_list[11]))

labels_list.append(('brainstem', *tmp_list[10]))
labels_list.append(('parotids', *tmp_list[10]))
labels_list.append(('all_maps', *tmp_list[10]))

for DATASET_REG_NAME, OAR_KEY, OAR_VALUE in labels_list:
    print(f"{DATASET_REG_NAME}, {OAR_KEY}, {OAR_VALUE}")

In [ ]:
def get_possible_models(oar_key):
    possible_models = [folder_name for folder_name in os.listdir('./models') if oar_key in folder_name]    
    
    return possible_models

### loading models to CPU 

In [ ]:
models = dict()
for DATASET_REG_NAME, OAR_KEY, OAR_VALUE in labels_list:
    model_reg_name = f'{OAR_KEY}-{DATASET_REG_NAME}'
    
    # dataset loading
    data_path = f'./data/HaN_OAR_cut_{DATASET_REG_NAME}_reg'
    cut_full_res_dataset = HaNOarsDataset(data_path, size=50, load_images=False)
    cut_full_res_dataset.load_from_file(data_path)
    cut_full_res_dataset.set_output_label(OAR_VALUE)
    cut_full_res_dataset_obj = split_dataset(cut_full_res_dataset, train_size=40, valid_size=5, test_size=5)
    cut_train_dataset, cut_valid_dataset, cut_test_dataset = itemgetter(*['train_dataset', 'valid_dataset', 'test_dataset'])(cut_full_res_dataset_obj)
    
    epoch = 1
    possible_models = get_possible_models(f"model3v2__cloud-{OAR_KEY}-{DATASET_REG_NAME}_reg")
    if len(possible_models) <= 0:
        print(f'{OAR_KEY} Model: No avaiable model')
        continue

    model_name = possible_models[0]
    print(f'{model_reg_name} Model: Loading model {model_name}')

    # loading model checkpoint
    cut_model_info = load_checkpoint_model_info(model_name, epoch, cut_train_dataset, cut_valid_dataset, cut_test_dataset, model_class=UNetV3v2)

    # moving model to cpu/cuda with eval mode
    cut_model_info['device'] = 'cpu'
    cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
    cut_model_info['model'].eval()
    cut_model_info['model'].disable_tensorboard_writing = True
    
    models[model_reg_name] = cut_model_info

In [ ]:
models.keys()

## Testing Eval vs Train Mode

testing iteration function

In [ ]:
cut_full_res_dataset.set_output_label(OARS_LABELS.PITUITARY)
cut_model_info = models[list(models.keys())[0]]
cut_model_info['device'] = get_device()
cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
cut_model_info['model'].disable_tensorboard_writing = True
    
model, model_name, optimizer, criterion = itemgetter('model', 'model_name', 'optimizer', 'criterion')(cut_model_info)
epochs, device, tensorboard_writer = itemgetter('epochs', 'device', 'tensorboard_writer')(cut_model_info)
train_dataloader, valid_dataloader, test_dataloader = itemgetter('train_dataloader',
                                                                 'valid_dataloader',
                                                                 'test_dataloader')(cut_model_info)
model.actual_epoch = 100

valid_loss, valid_dsc = iterate_model_v3v2(valid_dataloader, model, optimizer, criterion, device, is_eval=True)
print(valid_loss, valid_dsc)

cut_model_info['model'].disable_tensorboard_writing = True
cut_model_info['device'] = 'cpu'
cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])


In [ ]:
SHOW_DSC_INFO = True
if SHOW_DSC_INFO:
    info_per_organs_df = {}
    models_info = list()
    for DATASET_REG_NAME, OAR_KEY, OAR_VALUE in labels_list:
        model_name = f'{OAR_KEY}-{DATASET_REG_NAME}'
        
        if model_name not in models:
            print(f'{model_name} Model: No avaiable model')
            continue

        # getting model to gpu
        cut_model_info = models[model_name]
        cut_model_info['device'] = get_device()
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
        cut_model_info['model'].eval()
        cut_model_info['model'].disable_tensorboard_writing = True

        # preparing dataset for comparison
        # dataset loading
        data_path = f'./data/HaN_OAR_cut_{DATASET_REG_NAME}_reg'
        cut_full_res_dataset = HaNOarsDataset(data_path, size=50, load_images=False)
        cut_full_res_dataset.load_from_file(data_path)
        cut_full_res_dataset.set_output_label(OAR_VALUE)
        cut_full_res_dataset_obj = split_dataset(cut_full_res_dataset, train_size=40, valid_size=5, test_size=5)
        cut_train_dataset, cut_valid_dataset, cut_test_dataset = itemgetter(*['train_dataset', 'valid_dataset', 'test_dataset'])(cut_full_res_dataset_obj)
    

        # calculating dsc predictions        
        info_df, preds, rescaled_preds = get_threshold_info_df(
                                    model=cut_model_info['model'], 
                                    dataset=cut_full_res_dataset, 
                                    device=cut_model_info['device'], 
                                    train_indices=cut_train_dataset.indices, 
                                    valid_indices=cut_valid_dataset.indices, 
                                    test_indices=cut_test_dataset.indices,
                                    step=0.5,
                                    transform_input_fn=transform_input_with_registration)
        info_per_organs_df[model_name] = info_df

        # moving model back to cpu
        cut_model_info['device'] = 'cpu'
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])

        # parsing data
        best_threshold_col = 'thres_rescaled_dsc_0.50'
        train_tmp_df = info_df[info_df['is_train']][best_threshold_col]
        valid_tmp_df = info_df[info_df['is_valid']][best_threshold_col]
        train_dsc = train_tmp_df.mean()
        valid_dsc = valid_tmp_df.mean()
        print(f'{model_name} Model: DSC train {round(train_dsc, 4)} valid {round(valid_dsc, 4)}')

        models_info.append({
            'oar_key': OAR_KEY,
            'model_name': model_name,
            # Train
            'train_dsc_mean': train_dsc,
            'train_dsc_std': train_tmp_df.std(),
            'train_dsc_median': train_tmp_df.median(),
            'train_dsc_min': train_tmp_df.min(),
            'train_dsc_max': train_tmp_df.max(),
            # Valid
            'valid_dsc_mean': valid_dsc,
            'valid_dsc_std': valid_tmp_df.std(),
            'valid_dsc_median': valid_tmp_df.median(),
            'valid_dsc_min': valid_tmp_df.min(),
            'valid_dsc_max': valid_tmp_df.max(),
            # Both
            'train_valid_mean_delta': train_dsc - valid_dsc
        })

    models_info_df = pd.DataFrame(models_info)
    
    tmp_df = models_info_df[['oar_key', 'train_dsc_mean', 'train_dsc_std', 'valid_dsc_mean', 'valid_dsc_std']].copy()
    tmp_df['train_dsc_mean'] = (tmp_df['train_dsc_mean'] * 100).round(2)
    tmp_df['valid_dsc_mean'] = (tmp_df['valid_dsc_mean'] * 100).round(2)
    tmp_df['train_dsc_std'] = (tmp_df['train_dsc_std'] * 100).round(2)
    tmp_df['valid_dsc_std'] = (tmp_df['valid_dsc_std'] * 100).round(2)
    
    display(tmp_df.mean().round(2))
    display(tmp_df.round(2))
    display(tmp_df.sort_values(by=['train_dsc_std']).round(2))
    display(models_info_df.sort_values(by=['train_dsc_mean']).drop(columns=['model_name']).round(2))
    display(models_info_df.sort_values(by=['train_valid_mean_delta']).drop(columns=['model_name']).round(2))

In [ ]:
if SHOW_DSC_INFO:
    tmp_column = 'is_train'
    
    try:
        print('OARS_LABELS.PAROTID_GLAND_R')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PAROTID_GLAND_R]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass

    try:   
        print('OARS_LABELS.PAROTID_GLAND_L')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PAROTID_GLAND_L]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass
        
    try: 
        print('OARS_LABELS.OPT_NERVE_L')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.OPT_NERVE_L]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass
        
    try: 
        print('OARS_LABELS.PITUITARY')
        tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PITUITARY]]
        display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    except:
        pass

# Predictions merging and checking

In [ ]:
filter_labels_dict = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels_dict['SPINAL_CORD']

cut_full_res_dataset.set_output_label(filter_labels_dict)
preview_dataset(cut_full_res_dataset, preview_index=35)

In [ ]:
PARSE_CUT_DATASET = True
if PARSE_CUT_DATASET:
    from collections import defaultdict
    
    prediction_threshold = 0.5
    cut_dataset_predictions = defaultdict(lambda: defaultdict(lambda: np.zeros(cut_full_res_dataset[0][0][0].shape)))
    
    # for each label
    for label_index, val in enumerate(labels_list[:]):
        DATASET_REG_NAME, OAR_KEY, OAR_VALUE = val
        model_name = f'{OAR_KEY}-{DATASET_REG_NAME}'
        
        # loading model
        if model_name not in models:
            print(f'{label_index+1}/{len(labels_list)}: {model_name} Model: No avaiable model')
            continue
        print(f'{label_index+1}/{len(labels_list)}: {model_name} Model: got model {datetime.datetime.now()}')

        # getting model to gpu
        cut_model_info = models[model_name]
        cut_model_info['device'] = get_device()
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
        cut_model_info['model'].eval()
        cut_model_info['model'].disable_tensorboard_writing = True

        # for label in whole dataset
        for index in range(len(cut_full_res_dataset)):
            prediction, rescaled_pred = get_rescaled_pred(cut_model_info['model'], cut_full_res_dataset, 
                                                          cut_model_info['device'], index, use_only_one_dimension=False)
    
            cut_dataset_predictions[index][OAR_VALUE] = prediction[0]
            # extended_cut_full_res_dataset.data_list[index][label_index + 1] = prediction
            # extended_cut_full_res_dataset.data_list[index][label_index + 1] = ((rescaled_pred > prediction_threshold) * 1).astype(np.int8)

        # moving model back to cpu
        cut_model_info['device'] = 'cpu'
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])

In [ ]:
filter_labels_dict = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels_dict['SPINAL_CORD']

cut_full_res_dataset.set_output_label(filter_labels_dict)
preview_dataset(cut_full_res_dataset, preview_index=35)

In [ ]:
if PARSE_CUT_DATASET:
    def custom_preview_dataset(dataset, predictions, preview_index=0, show_hist=False, use_transform=False):
        data, label = dataset.get_raw_item_with_label_filter(preview_index)  # equivalent dataset[preview_index]
        if use_transform:
            transform = get_dataset_transform()
            data, label = transform_input(data, label, transform)

        prediction = predictions[preview_index]
        max_channels = label.shape[0]
        max_slices = label.shape[1]

        print(f'data max {data.max()}, min {data.min()}')
        print(f'label max {label.max()}, min {label.min()}')
        print(f'data {data.shape}, label {label.shape}')

        def f(slice_index, label_channel):
            plt.figure(figsize=(20, 10))
            plt.subplot(1, 3, 1)
            plt.imshow(data[0, slice_index], cmap="gray")
            plt.title('data')
            plt.subplot(1, 3, 2)
            plt.imshow(prediction[label_channel+1][slice_index])
            plt.title(f'prediction {label_channel}')
            plt.subplot(1, 3, 3)
            plt.imshow(label[label_channel, slice_index])
            plt.title(f'label {label_channel}')
            plt.show()

            if show_hist:
                plt.figure(figsize=(20, 10))
                plt.subplot(1, 2, 1)
                plt.hist(data.flatten(), 128)
                plt.subplot(1, 2, 2)
                plt.hist(label.flatten(), 128)
                plt.show()

        sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)
        labelChannelSlider = widgets.IntSlider(min=0, max=max_channels - 1, step=1, value=(max_channels - 1) / 2)
        ui = widgets.VBox([widgets.HBox([sliceSlider, labelChannelSlider])])
        out = widgets.interactive_output(f, {'slice_index': sliceSlider, 'label_channel': labelChannelSlider})
        # noinspection PyTypeChecker
        display(ui, out)
    
    index = cut_valid_dataset.indices[3]
    index = 35
    custom_preview_dataset(cut_full_res_dataset, cut_dataset_predictions, preview_index=index)